In [0]:
%run "/Workspace/Users/silvester.g@diggibyte.com/DataBricks_Assignment/src/question1/source_to_bronze/utils"

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS assignment.gold")

In [0]:

from pyspark.sql.functions import sum, avg, count, desc


employee_df = spark.read.table("assignment.Employee_info.dim_employee")

salary_per_dept = employee_df.groupBy("department_id") \
    .agg(sum("salary").alias("total_salary")) \
    .orderBy(desc("total_salary"))

emp_count = employee_df.groupBy("department_id", "country_id") \
    .agg(count("employee_id").alias("employee_count"))

dept_country = employee_df.select("department_id", "country_id").distinct()

avg_age = employee_df.groupBy("department_id") \
    .agg(avg("age").alias("avg_age"))

salary_per_dept = add_load_date(salary_per_dept)
emp_count = add_load_date(emp_count)
dept_country = add_load_date(dept_country)
avg_age = add_load_date(avg_age)

salary_per_dept.write.format("delta").mode("overwrite").saveAsTable("assignment.gold.fact_salary")
emp_count.write.format("delta").mode("overwrite").saveAsTable("assignment.gold.fact_emp_count")
dept_country.write.format("delta").mode("overwrite").saveAsTable("assignment.gold.fact_dept_country")
avg_age.write.format("delta").mode("overwrite").saveAsTable("assignment.gold.fact_avg_age")

# print("✅ Data successfully written as Delta tables in gold database")

In [0]:
%sql
select * from assignment.gold.fact_avg_age;

In [0]:

select * from assignment.gold.fact_salary